In [1]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhijitdeshpande/Library/Application Support/sagemaker/config.yaml


In [2]:
ROLE='arn:aws:iam::720332985926:role/service-role/AmazonSageMaker-ExecutionRole-20240705T185967'
SOURCE_DIR='/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/src'

In [3]:
sagemaker_session = sagemaker.Session()

In [4]:
model_uri = 's3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-19-23-24-05-737/output/model.tar.gz'

In [6]:
huggingface_model = HuggingFaceModel(
    entry_point='inference.py',
    model_data=model_uri,
    source_dir=SOURCE_DIR,
    role=ROLE,
    transformers_version='4.49.0',
    py_version='py312',
    pytorch_version='2.6.0',
    sagemaker_session=sagemaker_session
    )

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge'
    )


In [ ]:
predictor.predict({"inputs": "Classify the intent:  Can I cacnel my ticket?"})


In [7]:
transformer = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.m5.xlarge',
    strategy="SingleRecord",
    output_path='s3://sagemaker-us-east-1-720332985926/transformer/output/',
    accept="application/json",
    assemble_with="Line"
)

In [ ]:
transformer.transform(
    data="s3://gen-ai-repository/finetuning/flan-t5/data/flan-t5-test.jsonl",
    content_type="application/json",
    split_type='Line',
    wait=True	
)

INFO:sagemaker:Creating transform job with name: huggingface-pytorch-inference-2025-06-22-14-25-07-216


2025-06-22T14:29:51,013 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.12/site-packages
Current directory: /
Temp directory: /tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 3934 M
Python executable: /opt/conda/bin/python3.12
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /
Initial Models: model=/opt/ml/model
Log dir: null
2025-06-22T14:29:51,013 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.12/site-packages
Current directory: /
Temp directory: /tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 3934 M
Python executable: /opt/conda/bin/python3.12
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /
Initial Models: model=/opt/ml/model
Log dir: null
Metrics dir: null
Netty threads: 0
Netty client threads: 0
Default workers per model: 4
Bl

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 transformer.transform(                                                                       │
│   2 │   data="s3://gen-ai-repository/finetuning/flan-t5/data/flan-t5-test.jsonl",                │
│   3 │   content_type="application/jsonlines",                                                    │
│   4 │   split_type='Line'                                                                        │
│                                                                                                  │
│ /Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/sagemaker/work │
│ flow/pipeline_context.py:346 in wrapper                                                          │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/sagemaker/tran │
│ sformer.py:318 in transform                                                                      │
│                                                                                                  │
│   315 │   │   )                                                                                  │
│   316 │   │                                                                                      │
│   317 │   │   if wait:                                                                           │
│ ❱ 318 │   │   │   self.latest_transform_job.wait(logs=logs)                                      │
│   319 │                                                                                          │
│   320 │   def transform_with_monitoring(                                                         │
│   321 │   │   self,                                                                              │
│                                                                                                  │
│ /Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/sagemaker/tran │
│ sformer.py:686 in wait                                                                           │
│                                                                                                  │
│   683 │                                                                                          │
│   684 │   def wait(self, logs=True):                                                             │
│   685 │   │   if logs:                                                                           │
│ ❱ 686 │   │   │   self.sagemaker_session.logs_for_transform_job(self.job_name, wait=True)        │
│   687 │   │   else:                                                                              │
│   688 │   │   │   self.sagemaker_session.wait_for_transform_job(self.job_name)                   │
│   689                                                      

In [10]:
checkpoint_path = '/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/output/checkpoint-1888'

In [11]:
checkpoint_path

'/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/output/checkpoint-1888'

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig
import torch
import glob, os

In [13]:
#load base model & wrap with peft
peft_config = PeftConfig.from_pretrained(checkpoint_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, checkpoint_path)


/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [14]:
merged_model = model.merge_and_unload()

In [15]:
save_path = '/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/output/final_merged_model'

In [16]:
merged_model.save_pretrained(save_path)

In [18]:
model_uri

's3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-19-23-24-05-737/output/model.tar.gz'